# Google CoLab και TensorFlow 2

Ο κώδικας που ακολουθεί βεβαιώνει ότι τρέχουμε το TensorFlow 2 στο Colab.

In [ ]:
# Detect Colab if present
try:
    from google.colab import drive
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"

# ResNet στο Keras

Όσο βαθύτερα είναι τα νευρωνικά δίκτυα, τόσο δυσκολότερη είναι η εκπαίδευσή τους. Η μάθηση υπολειπόμενων (residuals) εισήχθη ώστε να γίνει εφικτή η εκπαίδευση δικτύων που είναι σημαντικά βαθύτερα από προηγούμενες αρχιτεκτονικές.

Το ResNet (Residual Network) επαναδιατυπώνει ρητά τα επίπεδα ώστε να μαθαίνουν συναρτήσεις υπολειπόμενων με αναφορά στα επίπεδα εισόδου, αντί να μην έχουν καμία αναφορά. 

Στο ImageNet, αυτή η μέθοδος έδωσε αρχιτεκτονικές με βάθος μέχρι και 152 επίπεδα, 8 φορές παραπάνω από το VGG, οι οποίες ωστόσο είναι μικρότερης πολυπλοκότητας.

Ένα σύνολο τέτοιων δικτύων υπολειπόμενων πέτυχε σφάλμα 3.57% στο σύνολο ελέγχου του ImageNet. Έτσι κέρδισε την πρώτη θέση στο task ταξινόμισης του ILSVRC 2015.

Θα εφαρμόσουμε την ίδια τεχνική στο CIFAR-10 με 100 και 1000 επίπεδα. 

Το ResNet εισήχθη στη δημοσίευση:

* K. He, X. Zhang, S. Ren, and J. Sun. [Deep residual learning for image recognition](https://arxiv.org/abs/1512.03385). arXiv preprint arXiv:1512.03385,2015.

Τί είναι ένα υπολειπόμενο;

* [Residual](https://www.merriam-webster.com/dictionary/residual): an internal aftereffect of experience or activity that influences later behavior

Για να υλοποιήσουμε το ResNet, πρέπει να δώσουμε στο Keras την έννοια ενός μπλοκ υπολειπόμενων. Ουσιαστικά πρόκειται για δύο πυκνά επίπεδα με μια σύνδεση "skip" (ή σύνδεση υπολειπόμενου). 

Στην επόμενη εικόνα βλέπουμε μια σύνδεση υπολειπόμενου.

![Skip Layers](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/skip-layer.png "Skip Layers")



Τα μπλοκ υπολειπόμενων τυπικά χρησιμοποιούνται μαζί με συνελικτικά δίκτυα (CNNs). Αυτό μας επιτρέπει να δημιουργούμε πολύ βαθιά συνελικτικά νευρωνικά δίκτυα.

Το επόμενο σχήμα δείχνει την αρχιτεκτονική του ResNet σε σχέση με το VGG και ένα FFNN αντίστοιχου βάθους.

![ResNets](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/resnet.png "ResNets")





### Το σειριακό και το συναρτησιακό API του Keras


Τα περισσότερα νευρωνικά δίκτυα που έχουμε δημιουργίσει βασίζονται στα σειριακά αντικείμενα του Keras. 
Είδαμε σύντομα ωστόσο ότι μπορούμε να έχουμε και έναν άλλο τύπου αντικειμένου για το ResNet, το "Model".
Αυτοί είναι οι δύο [βασικοί τρόποι](https://keras.io/getting-started/functional-api-guide/) για να κατασκευάζουμε νευρωνικά στο Keras:

* [Σειριακό API](https://keras.io/getting-started/sequential-model-guide/) - Μια απλοποιημένη διεπαφή του Keras που υποστηρίζει τα περισσότερα μοντέλα όπου η πληροφορία ακολουθεί πρόσθια τροφοδότηση σειριακά από την είσοδο στην έξοδο. 
* [Συναρτησιακό API](https://keras.io/getting-started/functional-api-guide/) - Μια πιο σύνθετη διεπαφή, που επιτρέπει να κατασκευάζουμε νευρωνικά όπου τα επίπεδα επαναχρησιμοποιούνται, έχουμε πολλαπλά επίπεδα εισόδο ή επαναληπτικές συνδέσεις.

Είναι σημαντικό να τονίσουμε ότι αυτές οι δύο διεπαφές δεν αντιστοιχούν σε διαφορετικούς τύπους νευρωνικών. Αντί αυτού, είναι μέσα γα την κατασκευή νευρωνικών δικτύων στο Keras. Κάποια είδη νευρωνικών, όπως τα δίκτυα πρόσθιας τροφοδότησης, μπορούν να υλοποιηθούν και με τους δύο τρόπους. Ωστόσο, κάποια άλλα ήδη νευρωνικών, όπως το ResNet και τα GANs μπορούν να υλοποιηθούν μόνο με το συναρτησιακό API.



### Σύνολο δεδομένων CIFAR

Τα σύνολα δεδομένων [CIFAR-10 και CIFAR-100](https://www.cs.toronto.edu/~kriz/cifar.html) χρησιμοποιούνται συχνά στην κοινότητα των νευρωνικών δικτύων. Αποτελούσαν αρχικά σύνολα δεδομένων διαγωνισμών. 

Το CIFAR-10 είναι ένα σύνολο δεδομένων από εικόνες χαμηλής ανάλησης που ανήκουν σε 10 κλάσεις. Το CIFAR-100 περιέχει 100 κλάσεις σε ιεραρχικές κατηγορίες. 


In [ ]:
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
import numpy as np
import os

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Μπορούμε να δούμε δείγματα του συνόλου δεδομένων CIFAR με τον ακόλουθο κώδικα.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from six.moves import cPickle 

ROWS = 10

x = x_train.astype("uint8")

fig, axes1 = plt.subplots(ROWS,ROWS,figsize=(10,10))
for j in range(ROWS):
    for k in range(ROWS):
        i = np.random.choice(range(len(x)))
        axes1[j][k].set_axis_off()
        axes1[j][k].imshow(x[i:i+1][0])

Θα κατασκευάσουμε ένα ResNet και θα το εκπαιδεύσουμε στο CIFAR-10. Το επόμενο μπλοκ κώδικα ορίζει κάποιες σταθερές τιμές σχετικές με το πως θα κατασκευαστεί το δίκτυο.

In [ ]:
# Training parameters
BATCH_SIZE = 32  # orig paper trained all networks with batch_size=128
EPOCHS = 5 # είναι λίγα αλλά αργεί
USE_AUGMENTATION = True
NUM_CLASSES = np.unique(y_train).shape[0] # 10
COLORS = x_train.shape[3]

# Subtracting pixel mean improves accuracy
SUBTRACT_PIXEL_MEAN = True

# Model version
# Orig paper: version = 1 (ResNet v1), 
# Improved ResNet: version = 2 
# (ResNet v2)
VERSION = 1

# Computed depth from supplied model parameter n
if VERSION == 1:
    DEPTH = COLORS * 6 + 2
elif version == 2:
    DEPTH = COLORS * 9 + 2

Η επόμενη συνάρτηση υλοποιεί ένα πρόγραμμα μείωσης του ρυθμού μάθησης.

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

Ο κώδικας που ακολουθεί υλοποιεί ένα μπλοκ ResNet. Περιλαμβάνει δύο συνελικτικά επίπεδα και μια σύνδεση skip. Και οι δύο εκδοχές του ResNet, V1 και V2, χρησιμοποιούν αυτού του τύπου τα επίπεδα.

In [ ]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

### ResNet V1

* K. He, X. Zhang, S. Ren, and J. Sun. [Deep residual learning for image recognition](https://arxiv.org/abs/1512.03385). arXiv preprint arXiv:1512.03385,2015.

In [ ]:
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature 
    map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of 
    filters is
    doubled. Within each stage, the layers have the same number 
    filters and the same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            # first layer but not first stack
            if stack > 0 and res_block == 0:  
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            # first layer but not first stack
            if stack > 0 and res_block == 0:  
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tensorflow.keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

### ResNet V2

Μια δεύτερη εκδοχή του ResNet εισήχθη στο άρθρο που ακολουθεί. Αυτή η μορφή ResNet αποκαλείται κοινώς ResNet V2.

* He, K., Zhang, X., Ren, S., & Sun, J. (2016, October). [Identity mappings in deep residual networks](https://arxiv.org/abs/1603.05027). In European conference on computer vision (pp. 630-645). Springer, Cham.

Ο επόμενος κώδικας κατασκευάζει ένα δίκτυο ResNet V2. Η βασική διαφορά της έκδοσης "v2" σε σχέση με την "v1" είναι η χρήση [κανονικοποίησης δέσμης](https://arxiv.org/abs/1502.03167) πριν την τα βάρη του πυκνού επιπέδου.

In [ ]:
def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is 
    halved (downsampled) by a convolutional layer with 
    strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same
    number filters and the same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tensorflow.keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

Μετά από αυτούς τους ορισμούς, μπορούμε πλέον να τρέξουμε ένα ResNet.

In [ ]:
# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if SUBTRACT_PIXEL_MEAN:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = tensorflow.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = tensorflow.keras.utils.to_categorical(y_test, NUM_CLASSES)

# Create the neural network
if VERSION == 2:
    model = resnet_v2(input_shape=input_shape, depth=DEPTH)
else:
    model = resnet_v1(input_shape=input_shape, depth=DEPTH)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()


In [ ]:
import time

start_time = time.time()

# Prepare callbacks for model saving and for learning rate adjustment.
lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not USE_AUGMENTATION:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation 
        # (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, 
                        batch_size=BATCH_SIZE),
                        validation_data=(x_test, y_test),
                        epochs=EPOCHS, verbose=0, workers=1,
                        callbacks=callbacks, 
                        use_multiprocessing=False)
    
elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Το εκπαιδευμένο δίκτυο μπορεί πλέον να αποτιμηθεί στα δεδομένα εκπαίδευσης.

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])